In [1]:
import binascii
import serial
import os

In [2]:
os.system('sh ./stop_sys_ttyPS0.sh')

32512

In [3]:
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        #print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()

In [4]:
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

In [5]:
def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

In [6]:
def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break
        

In [7]:
import numpy as np
import cv2
import time
from PIL import Image
np.set_printoptions(threshold=np.inf)

In [8]:
def obj_recog():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2
    # jupyter调用摄像头的代码
    im = cv2.imread(orig_img_path)
    #cv2.imshow('t',im)
    #cv2.imgshow(im)
    width = im.shape[1]
    height = im.shape[0]
    rate = round(width*0.5)
    rate1 = round(height*0.2)
    half = round(height/2)
    ims = im[rate1:half,0:(width-rate)]
    ims_right = im[(half+1):(height-rate1),0:(width-rate)]
    lower_red = np.array([0, 43, 20]) # 红色范围低阈值
    upper_red = np.array([20, 255, 255]) # 红色范围高阈值
    lower_red2 = np.array([150, 43, 20]) # 红色范围低阈值
    upper_red2 = np.array([200, 255, 255]) # 红色范围高阈值
    hsv_img = cv2.cvtColor(ims, cv2.COLOR_BGR2HSV)
    mask_red = cv2.inRange(hsv_img, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask_red,mask_red2)
    # again for right side
    hsv_img_right = cv2.cvtColor(ims_right, cv2.COLOR_BGR2HSV)
    mask_red_right = cv2.inRange(hsv_img_right, lower_red, upper_red)
    mask_red_right2 = cv2.inRange(hsv_img_right, lower_red2, upper_red2)
    mask_right = cv2.bitwise_or(mask_red_right,mask_red_right2)
    
    #print(mask_red)
    #print(mask_red2)
    #print(mask)
    #print(np.sum(mask_red))
    #print(np.sum(mask_red2))
    #print(np.sum(mask))
    nl = np.sum(mask)
    nr = np.sum(mask_right)
    #print(number)
    number = nl + nr
    
    h1 = height - rate1*2
    w1 = width - rate
    S = h1*w1
    print("nr",nr/255)
    print("nl",nl/255)
    test = nr/255 - nl/255
    print(test )
    print(number/S/255)
    number = (number/S/255)
    
    yu = 2000
    
    if(number < 0.01):
        kind =  0
    elif ((nr/255 - nl/255) >= yu): # yu zhi
        kind = -1
    elif ((nl/255 - nr/255) > yu):
        kind = 1
    else:
        kind = 2
    #if (number>0.01 && number<0.03 && number>0.5):
        #kind = 

    return kind



In [ ]:
start = time.time()
now = time.time() 
flag = 0
keep_right = 0
while(True):
    
    now = time.time()    
    kind = obj_recog()
    print("flag",flag)
    print("kind",kind)
    if(kind == 2): # red occurs, not know right or left
        #身体向右move，直到给定范围内没有红色区域
        keep_right = 1
        if(flag  == 0):
#             run_action('Stand')
#             wait_req()
            run_action('Right3move')
            wait_req()
        elif(flag == 1):    
#             run_action('Stand')
#             wait_req()
            run_action('Left3move')
            wait_req()
        
    elif kind == 1:
        flag = 0
        run_action('Right3move')
        wait_req()       
        
    elif kind == -1:  
        flag = 1
        run_action('Left3move')
        wait_req()   
        
    elif kind == 0:

            run_action('fastForward03')
            wait_req()
time.sleep(0.1)


In [ ]:
orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
# !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2> /dev/null
!fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2

img = Image.open(orig_img_path)
kind = obj_recog()
print(kind)
img